In [1]:
import pandas as pd
from datetime import datetime

pasta = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais"
pasta_bi = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos para BI"

RAIOS = ["1.5km", "5km", "10km", "20km"]

# Estacoes

In [2]:
cimehgo = pd.read_excel(fr"{pasta}\CIMEHGO_FINAL.xlsx")
inmet   = pd.read_excel(fr"{pasta}\INMET_FINAL.xlsx")

cimehgo_p = cimehgo.copy()
inmet_p = inmet.copy()

In [3]:
cimehgo_p = cimehgo_p.rename(columns={
    "ESTAÇÃO": "Estacao",
    "DATA/HORA(LOCAL)": "Data",
    "CHUVA(MM)": "Chuva_mm"
})

inmet_p = inmet_p.rename(columns={
    "ESTACAO": "Estacao",
    "Data": "Data",
    "Chuva(mm)": "Chuva_mm"
})

# Converte chuva para número
inmet_p["Chuva_mm"]   = pd.to_numeric(inmet_p["Chuva_mm"], errors="coerce").fillna(0)
cimehgo_p["Chuva_mm"] = pd.to_numeric(cimehgo_p["Chuva_mm"], errors="coerce").fillna(0)

# Ajuste da coluna Hora no INMET
inmet_p["Hora"] = pd.to_datetime(inmet_p["Hora"], format="%H:%M", errors="coerce").dt.time
inmet_p["Data"] = pd.to_datetime(inmet_p["Data"], errors="coerce")
inmet_p["Data"] = inmet_p.apply(lambda r: pd.Timestamp.combine(r["Data"], r["Hora"]), axis=1)

# Seleção final
cimehgo_p = cimehgo_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]
inmet_p   = inmet_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]

final_est = pd.concat([cimehgo_p, inmet_p], ignore_index=True)

# Salva para BI
final_est.to_csv(fr"{pasta_bi}\DADOS_UNIFICADOS_ESTACOES.csv", index=False, encoding="latin1")


# Raio

In [4]:
for raio in RAIOS:

    print(f"\n=== PROCESSANDO RAIO {raio} ===")

    # --- Carregar arquivos do raio ---
    lp = pd.read_excel(fr"{pasta}\Locais_por_Data_GEOCODED_FINAL_{raio}.xlsx")
    rp = pd.read_excel(fr"{pasta}\RELATOS_GEOCODED_FINAL_{raio}.xlsx")
    bp = pd.read_excel(fr"{pasta}\BOMBEIROS_GEOCODED_FINAL_{raio}.xlsx")

    # --- Padronizar nomes ---
    locais = lp.copy()
    relatos = rp.copy()
    bombeiros = bp.copy()

    if "ENDEREÇO" in bombeiros.columns:
        bombeiros = bombeiros.rename(columns={"ENDEREÇO": "LOCAL"})

    locais    = locais[[ "DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE" ]]
    relatos   = relatos[[ "DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE" ]]
    bombeiros = bombeiros[[ "DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE" ]]

    locais["Fonte"] = "Noticias"
    relatos["Fonte"] = "Relatos"
    bombeiros["Fonte"] = "Bombeiros"

    # -----------------------------------------------------------
    # Conversão de datas (SEM FUNÇÃO)
    # -----------------------------------------------------------
    def parse_date(x):
        if pd.isna(x):
            return pd.NaT
        s = str(x).strip()
        if "," in s:
            partes = s.split(",", 1)
            if len(partes) == 2:
                s = partes[1].strip()
        try:
            return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
        except:
            pass
        try:
            return datetime.strptime(s, "%B %d %Y")
        except:
            pass
        try:
            return datetime.strptime(s, "%B %d, %Y")
        except:
            pass
        return pd.NaT

    locais["DATA"] = locais["DATA"].apply(parse_date)
    relatos["DATA"] = relatos["DATA"].apply(parse_date)
    bombeiros["DATA"] = bombeiros["DATA"].apply(parse_date)

    # --- Junta tudo ---
    final_ocor = pd.concat([locais, relatos, bombeiros], ignore_index=True)

    # -----------------------------------------------------------
    # Merge com dados de chuva
    # -----------------------------------------------------------
    df_alag = final_ocor.copy()
    df_chuva = final_est.copy()

    df_chuva["Data"] = pd.to_datetime(df_chuva["Data"])
    df_chuva["data_dia"] = df_chuva["Data"].dt.date

    df_alag["DATA"] = pd.to_datetime(df_alag["DATA"])
    df_alag["data_dia"] = df_alag["DATA"].dt.date

    df_alag = df_alag.rename(columns={"Estação atribuída": "Estacao"})

    estacoes_validas = set(df_chuva["Estacao"].unique())
    df_alag_valid = df_alag[df_alag["Estacao"].isin(estacoes_validas)]

    df_expandido = pd.merge(
        df_alag_valid,
        df_chuva,
        on=["Estacao", "data_dia"],
        how="left"
    )

    # -----------------------------------------------------------
    # Estatísticas finais
    # -----------------------------------------------------------
    estatisticas_eventos = (
        df_expandido
        .groupby(["LOCAL", "DATA", "Estacao"])["Chuva_mm"]
        .agg(["min", "max", "mean"])
        .reset_index()
    )

    # -----------------------------------------------------------
    # Salva resultados para o raio
    # -----------------------------------------------------------
    df_expandido.to_excel(fr"{pasta_bi}\DADOS_EXPANDIDOS_{raio}.xlsx", index=False)
    estatisticas_eventos.to_excel(fr"{pasta_bi}\INDICADOR_{raio}.xlsx", index=False)

    print(f"✓ Concluído para {raio}")



=== PROCESSANDO RAIO 1.5km ===


C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it 

✓ Concluído para 1.5km

=== PROCESSANDO RAIO 5km ===


C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it 

✓ Concluído para 5km

=== PROCESSANDO RAIO 10km ===


C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it 

✓ Concluído para 10km

=== PROCESSANDO RAIO 20km ===


C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22820\2093213361.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it 

✓ Concluído para 20km
